## 1. MongoDB에 크롤링 데이터 지정하기
* Database, Collection 생성
* db : movie_db, col : actor_col
* data/actors.json file을 읽어서 처리하기

In [2]:
# warning 감추기
import warnings
warnings.filterwarnings(action='ignore')

In [12]:
import pymongo
import json

# 1. connection 생성
conn = pymongo.MongoClient('localhost',27017)
print(conn)

# 2. database 생성
movie = conn.movie_db
print(movie)
print(movie.name)

# 3. collection 생성
actors = movie_db.actor_col
print(actors)

# 4.json file open
with open('data/actors.json','r',encoding='utf-8') as file:
    json_data = json.load(file)
    actors.insert_many(json_data)

# document count
print(actors.estimated_document_count())

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'movie_db')
movie_db
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'movie_db'), 'actor_col')
1903


In [13]:
for doc in actors.find():
    print(doc)

{'_id': ObjectId('5f28b6276831af9fb858a557'), '배우이름': '이병헌', '흥행지수': 45760, '출연영화': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다'], '랭킹': 1, '다른 이름': 'Byung-hun Lee;BH Lee', '직업': '배우', '생년월일': '1970-07-12', '성별': '남', '홈페이지': 'http://www.leebyunghun.kr/', '신장/체중': '177cm, 72kg', '학교': '한양대학교 불어문학과', '취미': '모자수집, 여행', '특기': '태권도, 스노우보드, 수영, 팔씨름'}
{'_id': ObjectId('5f28b6276831af9fb858a558'), '배우이름': '하정우', '흥행지수': 31124, '출연영화': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨'], '랭킹': 2, '다른 이름': '김성훈; 河正宇', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': 'https://www.facebook.com/ft.hajungwoo', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영'}
{'_id': ObjectId('5f28b6276831af9fb858a559'), '배우이름': '마동석', '흥행지수': 19019, '출연영화': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연'], '랭킹': 3, '다른 이름': 'Ma Dongseok', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': 'https://www.instagram.com/madongseok_/https://twitter.co

### 1. sort

In [39]:
# 기본적으로 ascending
docs = actors.find({}).sort('배우이름')
# descending
docs = actors.find({}).sort('배우이름',pymongo.DESCENDING)
# sort by multiple field
docs = actors.find({}).sort([('배우이름',pymongo.ASCENDING),('랭킹',pymongo.DESCENDING)])
docs = actors.find({'학교':{'$regex':'한국예술'}}).sort('배우이름')
for idx,actor in enumerate(docs,1):
    print(idx,actor['배우이름'],actor['랭킹'],actor['흥행지수'],actor['학교'])

1 김동욱 1170 1 한국예술종합학교 연극원
2 김민재 91 3220 한국예술종합학교
3 김중기 1311 0 한국예술종합학교 연기과
4 박세준 882 3 한국예술종합학교 연기과 학사
5 박정민 7 14534 한국예술종합학교 영상원 연극원 연기과
6 서은아 522 30 한국예술종합학교 연기과 학사
7 신현빈 230 695 한국예술종합대학교 미술사이론학
8 유선 241 643 한국예술종합학교 연극원
9 이동규 1855 0 한국예술종합학교 연극영화과
10 이선균 152 1543 한국예술종합학교 연극
11 이승희 211 846 한국예술종합학교 연극원 연기과
12 이제훈 672 8 한국예술종합학교 연극원 연기과
13 정소민 266 477 한국예술종합학교 연극원 연기과
14 조현철 654 9 한국예술종합학교 예술사
15 한예리 1190 1 한국예술종합학교


### 2. exists
* 필드값이 존재하는 경우에 한해서 검색

In [27]:
# '특기' 필드가 존재하는 경우와 존재하지 않는 경우 count
print(actors.find({'특기':{'$exists':True}}).count())
print(actors.find({'특기':{'$exists':False}}).count())

171
1732


In [33]:
# '생년월일' 이 있는 배우의 이름과 생년월일
# find({},{'배우이름':1,'생년월일':1,'_id':0})
docs = actors.find({},{'배우이름':1,'생년월일':1,'_id':0})
docs = actors.find({'생년월일':{'$exists':True}})
for actor in docs:
    print(actor['배우이름'],actor['생년월일'])

이병헌 1970-07-12
하정우 1978-03-11
마동석 1971-03-01
전혜진 1976-08-10
이성민 1968-10-15
김아중 1982-10-16
박정민 1987-02-25
강기영 1983-10-14
조진웅 1976-03-03
조정석 1980-12-26
배수지 1994-10-10
유아인 1986-10-06
임윤아 1990-05-30
고두심 1951-05-22
장기용 1992-08-07
권상우 1976-08-05
곽도원 1974-00-00
유해진 1969-01-04
최유화 1985-10-10
박인환 1945-01-06
정해인 1988-04-01
정유미 1983-01-18
김지영 1974-09-07
류준열 1986-09-25
조우진 1979-01-16
공유 1979-07-10
강동원 1981-01-18
김종구 1955-00-00
박지환 1980-09-05
이정현 1980-02-07
김상중 1965-08-06
이희준 1979-06-29
김미경 1963-10-14
허성태 1977-10-20
박신혜 1990-02-18
이경영 1960-12-12
허준호 1964-03-03
권해효 1965-11-06
황효은 1978-06-04
이레 2006-03-12
이하늬 1983-03-02
김래원 1981-03-19
공민정 1986-09-30
성유빈 2000-07-25
우도환 1992-07-12
공효진 1980-04-04
최성은 1996-06-17
박해준 1976-06-14
윤경호 1980-07-05
조한철 1973-05-13
김무열 1982-05-22
김남길 1980-03-13
성동일 1967-04-27
류승범 1980-08-09
박성연 1975-06-24
김희원 1971-01-10
정민성 1975-07-07
정준호 1970-10-01
강신일 1960-11-26
정웅인 1971-01-20
설경구 1968-05-01
최덕문 1970-00-00
안재홍 1986-03-31
이재인 2004-02-06
김성철 1991-12-31
구교환 1982-12-14
황우슬혜 1979-08

안서현 2004-01-12
조민기 1966-11-05
김혜은 1973-03-01
김영진 1982-07-08
허기호 1952-03-28
박정범 1976-00-00
문희라 1990-03-26
도용구 1957-01-08
김재영 1988-09-30
이지현 1984-08-24
정기섭 1974-04-16
김도엽 2005-12-24
진세연 1994-02-15
정기섭 1974-04-16
김도엽 2005-12-24
진세연 1994-02-15
박성우 1988-06-07
백선혁 1988-04-02
김서원 1979-07-10
한지은 1990-06-03
서강준 1993-10-12
이일웅 1942-07-05
서강준 1993-10-12
이일웅 1942-07-05
노민우 1986-05-29
박정자 1942-03-12
이승신 1969-10-22
렌 1995-11-03
조윤희 1982-10-13
정윤겸 1992-09-04
신민아 1984-04-05
고동업 1961-02-15
김수로 1973-05-07
이일섭 1953-12-17
김수로 1973-05-07
지진희 1971-06-24
전고운 1985
지진희 1971-06-24
전고운 1985
이선호 1960-05-05
이환 1979-09-15
이영석 1959-00-00
송옥숙 1960-08-14
한진희 1949-03-14
김애경 1950-02-12
황민현 1995-08-09
박영수 1977-00-00
천민희 1987-08-27
김선아 1975-10-01
조지환 1978-04-02
황민현 1995-08-09
박영수 1977-00-00
천민희 1987-08-27
이재준 1990-10-20
이대연 1964-11-13
조상건 1946-00-00
차순형 1987-08-24
이찬희 1993-10-18
유퉁 1957-05-19
김민교 1974-08-05
강태우 1989-03-25
김금화 1931-08-18
김현빈 2004-04-19
유퉁 1957-05-19
김민교 1974-08-05
강태우 1989-03-25
김금화 1931-08-18
유대형 1985-01-

In [ ]:
# '다른 이름'이 '조원준'인 배우의 이름, 랭킹, 다른이름을 출력하시오

In [44]:
docs = actors.find({'다른 이름':{'$exists':True},'다른 이름':'조원준'})
for actor in docs:
    print(actor['배우이름'],actor['랭킹'],actor['다른 이름'])

조진웅 9 조원준


### 3. 필드의 범위로 검색
* \\$gt, $gte
* \\$lt, \\$lte

In [47]:
# 흥행지수 10000 <= 흥행지수 <= 30000 배우를 검색
# count()
# 배우이름:1, 흥행지수:1,  출연영화:1, _id:0
filter = {'흥행지수':{'$gte':10000,'$lte':30000}}
fields = {'배우이름':1,'흥행지수':1,'출연영화':1,'_id':0}
print(actors.find(filter).count())
for actor in actors.find(filter,fields):
    print(actor)

19
{'배우이름': '마동석', '흥행지수': 19019, '출연영화': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연']}
{'배우이름': '전혜진', '흥행지수': 17976, '출연영화': ['백두산', '비스트', '뺑반', '불한당: 나쁜 놈들의 세상', '택시운전사']}
{'배우이름': '이성민', '흥행지수': 15213, '출연영화': ['남산의 부장들', '비스트', '미스터 주: 사라진 VIP', '부당거래', '공작', '마이 리틀 히어로']}
{'배우이름': '김아중', '흥행지수': 15208, '출연영화': ['나쁜 녀석들: 더 무비', '더 킹']}
{'배우이름': '박정민', '흥행지수': 14534, '출연영화': ['시동', '다만 악에서 구하소서', '타짜: 원 아이드 잭', '사냥의 시간', '그것만이 내 세상', '동주']}
{'배우이름': '강기영', '흥행지수': 13974, '출연영화': ['엑시트', '가장 보통의 연애', '퍼즐']}
{'배우이름': '조진웅', '흥행지수': 13472, '출연영화': ['광대들: 풍문조작단', '퍼펙트맨', '블랙머니', '장수상회', '사라진 시간', '명량']}
{'배우이름': '조정석', '흥행지수': 13192, '출연영화': ['엑시트', '형', '특종: 량첸살인기', '시간이탈자', '뺑반', '나의 사랑 나의 신부']}
{'배우이름': '배수지', '흥행지수': 12646, '출연영화': ['백두산', '건축학개론']}
{'배우이름': '유아인', '흥행지수': 12422, '출연영화': ['＃살아있다', '베테랑', '버닝', '국가부도의 날', '우아한 거짓말']}
{'배우이름': '임윤아', '흥행지수': 12343, '출연영화': ['엑시트']}
{'배우이름': '고두심', '흥행지수': 11496, '출연영화': ['엑시트']}
{'배우이름': '장기용', '흥행지수': 11406, 

### 4. or, nor (not or)

In [48]:
# 흥행지수가 >= 15000 이거나 랭킹 <= 20 인 배우이름과 출연영화 리스트 출력하기
filter = {'$or':[{'흥행지수':{'$gte':15000}},{'랭킹':{'$lte':20}}]}
fields = {'배우이름':1,'출연영화':1,'_id':0}
for actor in actors.find(filter,fields):
    print(actor)

{'배우이름': '이병헌', '출연영화': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다']}
{'배우이름': '하정우', '출연영화': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨']}
{'배우이름': '마동석', '출연영화': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연']}
{'배우이름': '전혜진', '출연영화': ['백두산', '비스트', '뺑반', '불한당: 나쁜 놈들의 세상', '택시운전사']}
{'배우이름': '이성민', '출연영화': ['남산의 부장들', '비스트', '미스터 주: 사라진 VIP', '부당거래', '공작', '마이 리틀 히어로']}
{'배우이름': '김아중', '출연영화': ['나쁜 녀석들: 더 무비', '더 킹']}
{'배우이름': '박정민', '출연영화': ['시동', '다만 악에서 구하소서', '타짜: 원 아이드 잭', '사냥의 시간', '그것만이 내 세상', '동주']}
{'배우이름': '강기영', '출연영화': ['엑시트', '가장 보통의 연애', '퍼즐']}
{'배우이름': '조진웅', '출연영화': ['광대들: 풍문조작단', '퍼펙트맨', '블랙머니', '장수상회', '사라진 시간', '명량']}
{'배우이름': '조정석', '출연영화': ['엑시트', '형', '특종: 량첸살인기', '시간이탈자', '뺑반', '나의 사랑 나의 신부']}
{'배우이름': '배수지', '출연영화': ['백두산', '건축학개론']}
{'배우이름': '유아인', '출연영화': ['＃살아있다', '베테랑', '버닝', '국가부도의 날', '우아한 거짓말']}
{'배우이름': '임윤아', '출연영화': ['엑시트']}
{'배우이름': '고두심', '출연영화': ['엑시트']}
{'배우이름': '장기용', '출연영화': ['나쁜 녀석들: 더 무비

In [71]:
# $nor
# 흥행지수 >= 15000, 성별 = 남
match = {'$nor':[{'흥행지수':{'$lte':15000}},{'성별':'여'}]}
fields = {'배우이름':1,'성별':1,'흥행지수':1,'_id':0}
print(actors.find(match).count())
for actor in actors.find(match,fields):
    print(actor)

4
{'배우이름': '이병헌', '흥행지수': 45760, '성별': '남'}
{'배우이름': '하정우', '흥행지수': 31124, '성별': '남'}
{'배우이름': '마동석', '흥행지수': 19019, '성별': '남'}
{'배우이름': '이성민', '흥행지수': 15213, '성별': '남'}


### 5. $sortByCount
* aggregate() 함수 사용

In [60]:
# 소속사별 count
match = [{'$sortByCount':'$소속사'}]
match = [{'$sortByCount':'$소속사'},{'$match':{'count':{'$gte':4}}}]
for actor in actors.aggregate(match):
    print(actor)

{'_id': None, 'count': 1715}
{'_id': '아티스트컴퍼니', 'count': 17}
{'_id': '싸이더스 HQ', 'count': 13}
{'_id': '나무액터스', 'count': 11}
{'_id': 'SM엔터테인먼트', 'count': 9}
{'_id': '열음엔터테인먼트', 'count': 7}
{'_id': '케이엠컬쳐(주)', 'count': 5}
{'_id': '지티비엔터테인먼트', 'count': 5}
{'_id': '노아엔터테인먼트', 'count': 5}
{'_id': 'JYP엔터테인먼트', 'count': 4}
{'_id': '㈜사람엔터테인먼트', 'count': 4}
{'_id': '(주)YG엔터테인먼트', 'count': 4}
{'_id': '키이스트', 'count': 4}
{'_id': '예당엔터테인먼트', 'count': 4}
{'_id': '올리브나인', 'count': 4}


### 6. list 검색
* 리스트 안에 들어 있는 내용을 검색하기 위해서는 순서와 값이 정확이 일치해야 함
* 순서에 상관없이 값으롼 찾으려면 $all 를 사용한다.

In [70]:
# 출연영화 리스트에 '반도'를 포함하는 경우
match = {'출연영화':'반도'}
# 출연영화 리스트에 '1987' 또는 '반도'를 포함하는 경우
match = {'$or':[{'출연영화':'1987'},{'출연영화':'반도'}]}
# 출연영화 리스트에 '남산의 부장들'과 '공작'을 포함하는 경우
match = {'출연영화':['남산의 부장들','공작']} #조건에 만족하지 않는다. '남산의 부장들' 다음에 바로 '공작' 이 나오지 않으면 출력 안됨
# $all 사용
match = {'출연영화':{'$all':['남산의 부장들','공작']}} 
# 출연 영화의 첫번째 요소가 '결백'인 경우
match = {'출연영화.0':'결백'}
# 출연 영화의 두번째 요소가 '밀양'인 경우
match = {'출연영화.1':'밀양'}
# 출연 영화의 갯수가 6편인 경우
match = {'출연영화':{'$size':6}}

fields = {'배우이름':1, '출연영화':1, '_id':0}

for actor in actors.find(match,fields):
    print(actor)

{'배우이름': '이병헌', '출연영화': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다']}
{'배우이름': '하정우', '출연영화': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨']}
{'배우이름': '마동석', '출연영화': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연']}
{'배우이름': '이성민', '출연영화': ['남산의 부장들', '비스트', '미스터 주: 사라진 VIP', '부당거래', '공작', '마이 리틀 히어로']}
{'배우이름': '박정민', '출연영화': ['시동', '다만 악에서 구하소서', '타짜: 원 아이드 잭', '사냥의 시간', '그것만이 내 세상', '동주']}
{'배우이름': '조진웅', '출연영화': ['광대들: 풍문조작단', '퍼펙트맨', '블랙머니', '장수상회', '사라진 시간', '명량']}
{'배우이름': '조정석', '출연영화': ['엑시트', '형', '특종: 량첸살인기', '시간이탈자', '뺑반', '나의 사랑 나의 신부']}
{'배우이름': '곽도원', '출연영화': ['남산의 부장들', '강철비2: 정상회담', '변호인', '아수라', '무뢰한', '범죄와의 전쟁 : 나쁜놈들 전성시대']}
{'배우이름': '유해진', '출연영화': ['봉오동 전투', '부당거래', '타짜', '베테랑', '말모이', '럭키']}
{'배우이름': '류준열', '출연영화': ['봉오동 전투', '리틀 포레스트', '뺑반', '돈', '택시운전사', '더 킹']}
{'배우이름': '조우진', '출연영화': ['봉오동 전투', '내부자들', '돈', '국가부도의 날', '어쩌다, 결혼', '창궐']}
{'배우이름': '김종구', '출연영화': ['엑시트', '사랑의 선물', '욕창', '시', '광해, 왕이 된 남자', '들개']}

### 7. limit()

In [78]:
# 직업이 가수이면서 흥행지수가 높은 상위 10명의 배우는?
for actor in actors.find({'직업':'가수'}).sort('흥행지수',pymongo.DESCENDING).limit(10):
    print(actor)

{'_id': ObjectId('5f28b6276831af9fb858a561'), '배우이름': '배수지', '흥행지수': 12646, '출연영화': ['백두산', '건축학개론'], '랭킹': 11, '다른 이름': '배수지;미쓰에이;', '직업': '가수', '생년월일': '1994-10-10', '성별': '여', '홈페이지': 'https://twitter.com/missa_suzyhttps://www.instagram.com/skuukzky/', '신장/체중': '166cm, 47kg', '학교': '서울 공연예술고등학교', '소속사': 'JYP엔터테인먼트'}
{'_id': ObjectId('5f28b6276831af9fb858a563'), '배우이름': '임윤아', '흥행지수': 12343, '출연영화': ['엑시트'], '랭킹': 13, '다른 이름': '소녀시대; girlsgeneration; girls generation', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': 'https://www.instagram.com/yoona__lim/', '소속사': 'SM엔터테인먼트'}
{'_id': ObjectId('5f28b6276831af9fb858a5d5'), '배우이름': '최민호', '흥행지수': 2008, '출연영화': ['장사리 : 잊혀진 영웅들', '인랑'], '랭킹': 127, '다른 이름': '珉豪; 샤이니; SHINee', '직업': '가수', '생년월일': '1991-12-09', '성별': '남', '신장/체중': '181cm', '학교': '건국대학교 예술학', '취미': '축구, 농구', '특기': '연기, 영어, 중국어', '소속사': 'SM엔터테인먼트'}
{'_id': ObjectId('5f28b6276831af9fb858a611'), '배우이름': 'RM', '흥행지수': 1061, '출연영화': ['브링 더 소울 : 더 무비'], '랭킹': 187, '다른 이름': '김남준

In [77]:
# '국가부도의 날' 에 출연한 배우를 흥행지수가 높은 순으로 10명을 출력하시오
# _id만 빼고
for actor in actors.find({'출연영화':'국가부도의 날'},{'_id':0}).sort('흥행지수',pymongo.DESCENDING).limit(10):
    print(actor)

{'배우이름': '유아인', '흥행지수': 12422, '출연영화': ['＃살아있다', '베테랑', '버닝', '국가부도의 날', '우아한 거짓말'], '랭킹': 12, '다른 이름': '엄홍식', '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '홈페이지': 'https://www.instagram.com/hongsick/https://twitter.com/seeksik', '신장/체중': '178cm, 60kg', '학교': '단국대학교 연극', '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상', '특기': '스노우보드, 수영'}
{'배우이름': '조우진', '흥행지수': 8725, '출연영화': ['봉오동 전투', '내부자들', '돈', '국가부도의 날', '어쩌다, 결혼', '창궐'], '랭킹': 26, '다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}
{'배우이름': '허준호', '흥행지수': 6739, '출연영화': ['천문: 하늘에 묻는다', '퍼펙트맨', '결백', '국가부도의 날', '인랑'], '랭킹': 41, '원어명': '許俊豪', '직업': '배우', '생년월일': '1964-03-03', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과', '취미': '만화책 보기', '특기': '야구, 농구', '소속사': '지티비엔터테인먼트'}
{'배우이름': '전배수', '흥행지수': 1761, '출연영화': ['＃살아있다', '광해, 왕이 된 남자', '기묘한 가족', '국가부도의 날'], '랭킹': 138, '직업': '배우', '생년월일': '1970-06-02', '성별': '남'}
{'배우이름': '송영창', '흥행지수': 1715, '출연영화': ['정직한 후보', '변호인', '베테랑', '말모이', '국가부도의 날', '해빙'], '랭킹': 140, '다른 이름': '宋永彰', '직업': 